In [1]:
import pandas as pd
import numpy as np
import pyspi
from pyspi.calculator import Calculator
import os
from copy import deepcopy
import glob
import mne_connectivity
import mne

In [45]:
# Initialise a base calculator
calc = Calculator(subset='fast')

ROI_lookup = {"proc-0": "Category_Selective",
              "proc-1": "GNWT",
              "proc-2": "IIT"}

Loading configuration file: /Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/pyspi/fast_config.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})...
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLasso'})...
Succesfully initialised SPI with identifier "cov_GraphicalLasso" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[2] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLassoCV'})...
Succesfully initialised SPI with identifier "cov_GraphicalLassoCV" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[3] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'LedoitWolf'})...
Succesfully initialised SPI with identifier "cov_LedoitWolf" and labels ['basic', 'unordered', '

Frequency minimum set to 0; overriding to 1e-5.


In [27]:
data_path="/Users/abry4213/data/Cogitate_MEG_challenge/derivatives/MEG_time_series/sub-CA103_ses-1_epochs"
face_relevant_non_target_epoch_1_IIT_meta_ROI_time_series = pd.read_csv(f"{data_path}/face_Relevant-non-target_1000ms_epoch1_IIT_meta_ROI.csv")
face_relevant_non_target_epoch_1_GNWT_meta_ROI_time_series = pd.read_csv(f"{data_path}/face_Relevant-non-target_1000ms_epoch1_GNWT_meta_ROI.csv")
face_relevant_non_target_epoch_1_CS_meta_ROI_time_series = pd.read_csv(f"{data_path}/face_Relevant-non-target_1000ms_epoch1_Category_Selective_meta_ROI.csv")

face_relevant_non_target_epoch_1_all_time_series = pd.concat([face_relevant_non_target_epoch_1_IIT_meta_ROI_time_series, face_relevant_non_target_epoch_1_GNWT_meta_ROI_time_series, face_relevant_non_target_epoch_1_CS_meta_ROI_time_series], axis=0)
subject_id = "sub-CA103"

face_relevant_non_target_epoch_1_all_time_series['duration'] = face_relevant_non_target_epoch_1_all_time_series['duration'].str.replace('ms', '').astype(int)
face_relevant_non_target_epoch_1_all_time_series['times'] = np.round(face_relevant_non_target_epoch_1_all_time_series['times']*1000)
face_relevant_non_target_epoch_1_all_time_series['times'] = face_relevant_non_target_epoch_1_all_time_series['times'].astype(int)

# Filter times >= 0
face_relevant_non_target_epoch_1_all_time_series = face_relevant_non_target_epoch_1_all_time_series.query('times >= 0')

# Assign stimulus as on if times < duration and off if times >= duration
face_relevant_non_target_epoch_1_all_time_series['stimulus'] = np.where(face_relevant_non_target_epoch_1_all_time_series['times'] < face_relevant_non_target_epoch_1_all_time_series['duration'], 'on', 'off')

In [49]:
all_epoch_files = [f for f in os.listdir(data_path) if "GNWT" in f]
meta_ROI_list = ["IIT", "GNWT", "Category_Selective"]

all_pyspi_results_for_this_subject_list = []

for csv_file in all_epoch_files:
    stimulus_type, relevance, duration, epoch_number = csv_file.split("_")[0:4]

    results_across_ROIs_list = []

    # Combine results across IIT, GNWT, and Category-Selective meta-ROIs
    for meta_ROI in meta_ROI_list:
        this_ROI_data = pd.read_csv(f"{data_path}/{stimulus_type}_{relevance}_{duration}_{epoch_number}_{meta_ROI}_meta_ROI.csv")
        this_ROI_data['duration'] = this_ROI_data['duration'].str.replace('ms', '').astype(int)
        this_ROI_data['times'] = np.round(this_ROI_data['times']*1000)
        this_ROI_data['times'] = this_ROI_data['times'].astype(int)

        # Filter times >= 0
        this_ROI_data = this_ROI_data.query('times >= 0')

        # Assign stimulus as on if times < duration and off if times >= duration
        this_ROI_data['stimulus'] = np.where(this_ROI_data['times'] < this_ROI_data['duration'], 'on', 'off')

        # Append to list
        results_across_ROIs_list.append(this_ROI_data)

    results_across_ROIs = pd.concat(results_across_ROIs_list, axis=0).reset_index()

    # Iterate over onset vs. offset
    for stimulus_presentation in ['on', 'off']:
        results_across_ROIs_with_this_stim = results_across_ROIs.query("stimulus==@stimulus_presentation")

        # Pivot so that the columns are meta_ROI and the rows are data
        df_wide = (results_across_ROIs_with_this_stim.filter(items=['times', 'meta_ROI', 'data'])
                        .reset_index()
                        .pivot(index='meta_ROI', columns='times', values='data'))

        # Make deepcopy of calc 
        calc_copy = deepcopy(calc)

        # Convert to numpy array
        TS_array = df_wide.to_numpy()

        # Load data 
        calc_copy.load_dataset(TS_array)
        calc_copy.compute()

        SPI_res = deepcopy(calc_copy.table)

        # Iterate over each SPI
        SPI_res.columns = SPI_res.columns.to_flat_index()

        SPI_res = SPI_res.rename(columns='__'.join).assign(meta_ROI_from = lambda x: x.index)
        SPI_res_long = SPI_res.melt(id_vars='meta_ROI_from', var_name='SPI__meta_ROI_to', value_name='value')

        SPI_res_long["SPI"] = SPI_res_long["SPI__meta_ROI_to"].str.split("__").str[0]
        SPI_res_long["meta_ROI_to"] = SPI_res_long["SPI__meta_ROI_to"].str.split("__").str[1]

        SPI_res_long = (SPI_res_long
                        .drop(columns='SPI__meta_ROI_to')
                        .query('meta_ROI_from != meta_ROI_to')
                        .assign(meta_ROI_from = lambda x: x['meta_ROI_from'].map(ROI_lookup),
                                meta_ROI_to = lambda x: x['meta_ROI_to'].map(ROI_lookup))
                        .filter(items=['SPI', 'meta_ROI_from', 'meta_ROI_to', 'value'])
                        .assign(stimulus_type = stimulus_type,
                                stimulus_presentation = stimulus_presentation,
                                relevance_type = relevance,
                                duration = duration,
                                epoch_number = epoch_number,
                                subject_ID = subject_id)
        )

        all_pyspi_results_for_this_subject_list.append(SPI_res_long)

all_pyspi_results_for_this_subject = pd.concat(all_pyspi_results_for_this_subject_list, axis=0).reset_index()

all_pyspi_results_for_this_subject.to_csv(f"{data_path}/pyspi_all_epochs_sub-CA103.csv", index=False)

Processing [None: phase_multitaper_mean_fs-1_fmin-0_fmax-0-5]:  33%|███▎      | 72/215 [00:11<01:10,  2.03it/s]Mean of empty slice
Processing [None: phase_multitaper_max_fs-1_fmin-0_fmax-0-5]:  33%|███▎      | 72/215 [00:11<01:10,  2.03it/s]    All-NaN slice encountered
Processing [None: sgc_nonparametric_mean_fs-1_fmin-0_fmax-0-5]:  73%|███████▎  | 158/215 [00:12<00:04, 12.05it/s]   Mean of empty slice
Processing [None: sgc_nonparametric_max_fs-1_fmin-0_fmax-0-5]:  76%|███████▋  | 164/215 [00:13<00:03, 14.12it/s]    All-NaN slice encountered
Processing [None: pec_orth_log_abs]: 100%|██████████| 215/215 [00:13<00:00, 15.90it/s]                                      
Processing [None: phase_multitaper_mean_fs-1_fmin-0_fmax-0-5]:  33%|███▎      | 70/215 [00:11<00:48,  3.00it/s]Mean of empty slice
Processing [None: phase_multitaper_max_fs-1_fmin-0_fmax-0-5]:  33%|███▎      | 70/215 [00:11<00:48,  3.00it/s]    All-NaN slice encountered
Processing [None: sgc_nonparametric_mean_fs-1_fmin-0_fm